In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 8
tree_depth = 12
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.198968887329102 | KNN Loss: 6.225536346435547 | BCE Loss: 1.9734323024749756
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.135589599609375 | KNN Loss: 6.225613117218018 | BCE Loss: 1.9099769592285156
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.183895111083984 | KNN Loss: 6.225164413452148 | BCE Loss: 1.9587311744689941
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.114386558532715 | KNN Loss: 6.22507905960083 | BCE Loss: 1.8893071413040161
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.159957885742188 | KNN Loss: 6.224706172943115 | BCE Loss: 1.9352517127990723
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.136695861816406 | KNN Loss: 6.224417686462402 | BCE Loss: 1.912278652191162
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.143320083618164 | KNN Loss: 6.2238640785217285 | BCE Loss: 1.9194557666778564
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.132482528686523 | KNN Loss: 6.223404407501221 | BCE Loss: 1.909078

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 5.453589916229248 | KNN Loss: 4.331330299377441 | BCE Loss: 1.122259497642517
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 5.370828628540039 | KNN Loss: 4.244254112243652 | BCE Loss: 1.1265745162963867
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 5.28737735748291 | KNN Loss: 4.171576499938965 | BCE Loss: 1.1158009767532349
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 5.168726921081543 | KNN Loss: 4.064449787139893 | BCE Loss: 1.1042771339416504
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 5.064929008483887 | KNN Loss: 3.9546024799346924 | BCE Loss: 1.1103267669677734
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 4.961371421813965 | KNN Loss: 3.8593170642852783 | BCE Loss: 1.1020543575286865
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 4.923951148986816 | KNN Loss: 3.8265035152435303 | BCE Loss: 1.0974478721618652
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 4.831162452697754 | KNN Loss: 3.720099925994873 | BCE Los

Epoch 21 / 500 | iteration 15 / 30 | Total Loss: 3.6977899074554443 | KNN Loss: 2.6339762210845947 | BCE Loss: 1.0638136863708496
Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 3.7353272438049316 | KNN Loss: 2.6816258430480957 | BCE Loss: 1.0537012815475464
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 3.670689105987549 | KNN Loss: 2.639291763305664 | BCE Loss: 1.0313974618911743
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 3.6490042209625244 | KNN Loss: 2.616891860961914 | BCE Loss: 1.0321123600006104
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 3.6471664905548096 | KNN Loss: 2.5883514881134033 | BCE Loss: 1.0588150024414062
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 3.649770498275757 | KNN Loss: 2.6246516704559326 | BCE Loss: 1.0251188278198242
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 3.6736395359039307 | KNN Loss: 2.6033623218536377 | BCE Loss: 1.070277214050293
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 3.6453888416290283 | KNN Loss: 2.597358226776123

Epoch 32 / 500 | iteration 5 / 30 | Total Loss: 3.618807077407837 | KNN Loss: 2.5717806816101074 | BCE Loss: 1.0470263957977295
Epoch 32 / 500 | iteration 10 / 30 | Total Loss: 3.569949150085449 | KNN Loss: 2.5322418212890625 | BCE Loss: 1.0377073287963867
Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 3.577805995941162 | KNN Loss: 2.5482277870178223 | BCE Loss: 1.0295783281326294
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 3.6160964965820312 | KNN Loss: 2.5657830238342285 | BCE Loss: 1.0503135919570923
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 3.622591495513916 | KNN Loss: 2.5976924896240234 | BCE Loss: 1.0248990058898926
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 3.5767431259155273 | KNN Loss: 2.555893898010254 | BCE Loss: 1.0208492279052734
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 3.610532760620117 | KNN Loss: 2.5846874713897705 | BCE Loss: 1.0258452892303467
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 3.701009511947632 | KNN Loss: 2.645540952682495 | 

Epoch 42 / 500 | iteration 25 / 30 | Total Loss: 3.599447250366211 | KNN Loss: 2.5760529041290283 | BCE Loss: 1.0233944654464722
Epoch 43 / 500 | iteration 0 / 30 | Total Loss: 3.576629161834717 | KNN Loss: 2.5417699813842773 | BCE Loss: 1.0348591804504395
Epoch 43 / 500 | iteration 5 / 30 | Total Loss: 3.612381935119629 | KNN Loss: 2.572984457015991 | BCE Loss: 1.0393973588943481
Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 3.5771279335021973 | KNN Loss: 2.5571000576019287 | BCE Loss: 1.0200278759002686
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 3.6161727905273438 | KNN Loss: 2.5712759494781494 | BCE Loss: 1.0448968410491943
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 3.5584521293640137 | KNN Loss: 2.5300369262695312 | BCE Loss: 1.0284152030944824
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 3.539503335952759 | KNN Loss: 2.492119789123535 | BCE Loss: 1.0473835468292236
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 3.569394588470459 | KNN Loss: 2.5309765338897705 |

...

Epoch 454 / 500 | iteration 25 / 30 | Total Loss: 3.488745927810669 | KNN Loss: 2.463747024536133 | BCE Loss: 1.0249989032745361
Epoch 455 / 500 | iteration 0 / 30 | Total Loss: 3.4412522315979004 | KNN Loss: 2.463918924331665 | BCE Loss: 0.9773333072662354
Epoch 455 / 500 | iteration 5 / 30 | Total Loss: 3.467306613922119 | KNN Loss: 2.4757308959960938 | BCE Loss: 0.9915756583213806
Epoch 455 / 500 | iteration 10 / 30 | Total Loss: 3.512146234512329 | KNN Loss: 2.493724822998047 | BCE Loss: 1.0184214115142822
Epoch 455 / 500 | iteration 15 / 30 | Total Loss: 3.498267650604248 | KNN Loss: 2.504296064376831 | BCE Loss: 0.9939714670181274
Epoch 455 / 500 | iteration 20 / 30 | Total Loss: 3.4648666381835938 | KNN Loss: 2.4375252723693848 | BCE Loss: 1.0273414850234985
Epoch 455 / 500 | iteration 25 / 30 | Total Loss: 3.4987854957580566 | KNN Loss: 2.466909170150757 | BCE Loss: 1.0318762063980103
Epoch 456 / 500 | iteration 0 / 30 | Total Loss: 3.4794301986694336 | KNN Loss: 2.469111204147

Epoch 465 / 500 | iteration 10 / 30 | Total Loss: 3.4834883213043213 | KNN Loss: 2.4896440505981445 | BCE Loss: 0.9938442707061768
Epoch 465 / 500 | iteration 15 / 30 | Total Loss: 3.5023272037506104 | KNN Loss: 2.4970390796661377 | BCE Loss: 1.0052881240844727
Epoch 465 / 500 | iteration 20 / 30 | Total Loss: 3.4285550117492676 | KNN Loss: 2.429630756378174 | BCE Loss: 0.9989243149757385
Epoch 465 / 500 | iteration 25 / 30 | Total Loss: 3.4695472717285156 | KNN Loss: 2.4654178619384766 | BCE Loss: 1.0041295289993286
Epoch 466 / 500 | iteration 0 / 30 | Total Loss: 3.4450631141662598 | KNN Loss: 2.422218084335327 | BCE Loss: 1.0228450298309326
Epoch 466 / 500 | iteration 5 / 30 | Total Loss: 3.459489107131958 | KNN Loss: 2.458935022354126 | BCE Loss: 1.000554084777832
Epoch 466 / 500 | iteration 10 / 30 | Total Loss: 3.4836134910583496 | KNN Loss: 2.457202911376953 | BCE Loss: 1.0264105796813965
Epoch 466 / 500 | iteration 15 / 30 | Total Loss: 3.495404005050659 | KNN Loss: 2.492884159

Epoch 475 / 500 | iteration 25 / 30 | Total Loss: 3.5007741451263428 | KNN Loss: 2.466007947921753 | BCE Loss: 1.0347661972045898
Epoch 476 / 500 | iteration 0 / 30 | Total Loss: 3.478994846343994 | KNN Loss: 2.447139263153076 | BCE Loss: 1.0318554639816284
Epoch 476 / 500 | iteration 5 / 30 | Total Loss: 3.5263655185699463 | KNN Loss: 2.5072953701019287 | BCE Loss: 1.0190701484680176
Epoch 476 / 500 | iteration 10 / 30 | Total Loss: 3.481199264526367 | KNN Loss: 2.435866355895996 | BCE Loss: 1.0453327894210815
Epoch 476 / 500 | iteration 15 / 30 | Total Loss: 3.447815179824829 | KNN Loss: 2.4393181800842285 | BCE Loss: 1.0084969997406006
Epoch 476 / 500 | iteration 20 / 30 | Total Loss: 3.46799373626709 | KNN Loss: 2.4431240558624268 | BCE Loss: 1.0248697996139526
Epoch 476 / 500 | iteration 25 / 30 | Total Loss: 3.462148427963257 | KNN Loss: 2.473351240158081 | BCE Loss: 0.988797128200531
Epoch 477 / 500 | iteration 0 / 30 | Total Loss: 3.4834818840026855 | KNN Loss: 2.47166061401367

Epoch 486 / 500 | iteration 10 / 30 | Total Loss: 3.4681904315948486 | KNN Loss: 2.44805908203125 | BCE Loss: 1.0201313495635986
Epoch 486 / 500 | iteration 15 / 30 | Total Loss: 3.4739770889282227 | KNN Loss: 2.4568309783935547 | BCE Loss: 1.017146110534668
Epoch 486 / 500 | iteration 20 / 30 | Total Loss: 3.452244997024536 | KNN Loss: 2.422790050506592 | BCE Loss: 1.0294549465179443
Epoch 486 / 500 | iteration 25 / 30 | Total Loss: 3.4274847507476807 | KNN Loss: 2.4373679161071777 | BCE Loss: 0.9901167750358582
Epoch 487 / 500 | iteration 0 / 30 | Total Loss: 3.5117275714874268 | KNN Loss: 2.470431089401245 | BCE Loss: 1.0412964820861816
Epoch 487 / 500 | iteration 5 / 30 | Total Loss: 3.469747543334961 | KNN Loss: 2.4631640911102295 | BCE Loss: 1.0065834522247314
Epoch 487 / 500 | iteration 10 / 30 | Total Loss: 3.432394504547119 | KNN Loss: 2.424036741256714 | BCE Loss: 1.0083577632904053
Epoch 487 / 500 | iteration 15 / 30 | Total Loss: 3.464578151702881 | KNN Loss: 2.436367988586

Epoch 496 / 500 | iteration 25 / 30 | Total Loss: 3.4983267784118652 | KNN Loss: 2.4660351276397705 | BCE Loss: 1.0322915315628052
Epoch 497 / 500 | iteration 0 / 30 | Total Loss: 3.459026336669922 | KNN Loss: 2.448936700820923 | BCE Loss: 1.010089635848999
Epoch 497 / 500 | iteration 5 / 30 | Total Loss: 3.456577777862549 | KNN Loss: 2.4369843006134033 | BCE Loss: 1.019593358039856
Epoch 497 / 500 | iteration 10 / 30 | Total Loss: 3.4633026123046875 | KNN Loss: 2.4450020790100098 | BCE Loss: 1.0183006525039673
Epoch 497 / 500 | iteration 15 / 30 | Total Loss: 3.49482798576355 | KNN Loss: 2.481560707092285 | BCE Loss: 1.0132672786712646
Epoch 497 / 500 | iteration 20 / 30 | Total Loss: 3.4193570613861084 | KNN Loss: 2.4318349361419678 | BCE Loss: 0.9875220656394958
Epoch 497 / 500 | iteration 25 / 30 | Total Loss: 3.443190097808838 | KNN Loss: 2.4283461570739746 | BCE Loss: 1.0148439407348633
Epoch 498 / 500 | iteration 0 / 30 | Total Loss: 3.477756977081299 | KNN Loss: 2.4426386356353

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 2.4856,  2.3745,  2.7505,  3.4871,  3.6530,  0.4236,  2.7899,  2.3621,
          2.5124,  2.1661,  2.3326,  2.4144,  0.6715,  1.7256,  1.4388,  1.7747,
          2.8992,  3.2629,  2.5353,  2.0810,  1.7299,  3.1325,  2.4733,  2.7931,
          2.7131,  1.8447,  1.3999,  1.1521,  1.0779,  0.2516, -0.5336,  1.1434,
          0.1360,  1.0365,  1.1041,  1.1722,  1.2571,  2.9358,  0.5095,  0.9384,
          0.7459, -0.5871, -0.1879,  2.4866,  2.4053,  0.7438, -0.2547, -0.0358,
          1.0731,  2.3665,  2.0010,  0.1006,  1.6326,  0.4142, -0.5522,  0.5195,
          1.5745,  1.3536,  1.5414,  1.1375,  0.4956,  1.0106,  0.2676,  1.3019,
          1.4890,  1.8231, -2.0561,  0.0387,  2.0633,  1.9456,  2.1997,  0.2808,
          1.5345,  2.7063,  1.9748,  1.0967,  0.2422,  0.7985,  0.2194,  1.6211,
          0.1200,  0.5925,  2.0239, -0.3642,  0.3667, -0.9796, -2.3407, -0.5386,
          0.2856, -1.8904,  0.4330, -0.1589, -0.9170, -0.7434,  0.4221,  1.2528,
         -1.2124, -0.8169,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [10]:
dataset_ = [d[0].cpu() for d in dataset]

In [11]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 79.45it/s]


In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [13]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [14]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [16]:
tensor_dataset = torch.stack(dataset_)

In [17]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [18]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [19]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [20]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [21]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [22]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [23]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [24]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [25]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [26]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [27]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [28]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [29]:
losses = []
accs = []
sparsity = []

In [30]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
layer 9: 0.0
layer 10: 0.0
Epoch: 00 | Batch: 000 / 003 | Total loss: 9.629 | Reg loss: 0.014 | Tree loss: 9.629 | Accuracy: 0.000000 | 3.81 sec/iter
Epoch: 00 | Batch: 001 / 003 | Total loss: 9.627 | Reg loss: 0.013 | Tree loss: 9.627 | Accuracy: 0.000000 | 3.735 sec/iter
Epoch: 00 | Batch: 002 / 003 | Total loss: 9.623 | Reg loss: 0.012 | Tree loss: 9.623 | Accuracy: 0.000000 | 3.564 sec/iter
Average sparseness: 0.9821428571428572
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
layer 9: 0.9821428571428573
layer 10: 0.9821428571428573
Epoch: 01 | Batch: 000 / 003 | Total loss: 9.623 | Reg loss: 0.003 | Tree loss: 9.623 | Accuracy: 0.000000 | 3.722

Epoch: 11 | Batch: 001 / 003 | Total loss: 9.586 | Reg loss: 0.004 | Tree loss: 9.586 | Accuracy: 0.234375 | 3.685 sec/iter
Epoch: 11 | Batch: 002 / 003 | Total loss: 9.583 | Reg loss: 0.004 | Tree loss: 9.583 | Accuracy: 0.229814 | 3.672 sec/iter
Average sparseness: 0.9821428571428572
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
layer 9: 0.9821428571428573
layer 10: 0.9821428571428573
Epoch: 12 | Batch: 000 / 003 | Total loss: 9.585 | Reg loss: 0.004 | Tree loss: 9.585 | Accuracy: 0.230469 | 3.687 sec/iter
Epoch: 12 | Batch: 001 / 003 | Total loss: 9.583 | Reg loss: 0.004 | Tree loss: 9.583 | Accuracy: 0.230469 | 3.685 sec/iter
Epoch: 12 | Batch: 002 / 003 | Total loss: 9.580 | Reg loss: 0.004 | Tree loss: 9.580 | Accuracy: 0.245342 | 3.673 sec/iter
Average sparseness: 0.98214285714

Epoch: 23 | Batch: 000 / 003 | Total loss: 9.562 | Reg loss: 0.006 | Tree loss: 9.562 | Accuracy: 0.234375 | 3.687 sec/iter
Epoch: 23 | Batch: 001 / 003 | Total loss: 9.562 | Reg loss: 0.006 | Tree loss: 9.562 | Accuracy: 0.226562 | 3.686 sec/iter
Epoch: 23 | Batch: 002 / 003 | Total loss: 9.556 | Reg loss: 0.006 | Tree loss: 9.556 | Accuracy: 0.245342 | 3.68 sec/iter
Average sparseness: 0.9821428571428572
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
layer 9: 0.9821428571428573
layer 10: 0.9821428571428573
Epoch: 24 | Batch: 000 / 003 | Total loss: 9.563 | Reg loss: 0.006 | Tree loss: 9.563 | Accuracy: 0.212891 | 3.687 sec/iter
Epoch: 24 | Batch: 001 / 003 | Total loss: 9.557 | Reg loss: 0.006 | Tree loss: 9.557 | Accuracy: 0.242188 | 3.686 sec/iter
Epoch: 24 | Batch: 002 / 003 | Tot

Average sparseness: 0.9821428571428572
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
layer 9: 0.9821428571428573
layer 10: 0.9821428571428573
Epoch: 35 | Batch: 000 / 003 | Total loss: 9.527 | Reg loss: 0.008 | Tree loss: 9.527 | Accuracy: 0.255859 | 3.685 sec/iter
Epoch: 35 | Batch: 001 / 003 | Total loss: 9.527 | Reg loss: 0.009 | Tree loss: 9.527 | Accuracy: 0.232422 | 3.684 sec/iter
Epoch: 35 | Batch: 002 / 003 | Total loss: 9.524 | Reg loss: 0.009 | Tree loss: 9.524 | Accuracy: 0.201863 | 3.68 sec/iter
Average sparseness: 0.9821428571428572
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.982142857

Epoch: 46 | Batch: 000 / 003 | Total loss: 9.456 | Reg loss: 0.011 | Tree loss: 9.456 | Accuracy: 0.214844 | 3.682 sec/iter
Epoch: 46 | Batch: 001 / 003 | Total loss: 9.435 | Reg loss: 0.011 | Tree loss: 9.435 | Accuracy: 0.253906 | 3.682 sec/iter
Epoch: 46 | Batch: 002 / 003 | Total loss: 9.422 | Reg loss: 0.011 | Tree loss: 9.422 | Accuracy: 0.232919 | 3.678 sec/iter
Average sparseness: 0.9821428571428572
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
layer 9: 0.9821428571428573
layer 10: 0.9821428571428573
Epoch: 47 | Batch: 000 / 003 | Total loss: 9.435 | Reg loss: 0.011 | Tree loss: 9.435 | Accuracy: 0.244141 | 3.682 sec/iter
Epoch: 47 | Batch: 001 / 003 | Total loss: 9.425 | Reg loss: 0.011 | Tree loss: 9.425 | Accuracy: 0.228516 | 3.682 sec/iter
Epoch: 47 | Batch: 002 / 003 | To

Average sparseness: 0.9821428571428572
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
layer 9: 0.9821428571428573
layer 10: 0.9821428571428573
Epoch: 58 | Batch: 000 / 003 | Total loss: 9.240 | Reg loss: 0.013 | Tree loss: 9.240 | Accuracy: 0.226562 | 3.682 sec/iter
Epoch: 58 | Batch: 001 / 003 | Total loss: 9.212 | Reg loss: 0.013 | Tree loss: 9.212 | Accuracy: 0.236328 | 3.681 sec/iter
Epoch: 58 | Batch: 002 / 003 | Total loss: 9.186 | Reg loss: 0.013 | Tree loss: 9.186 | Accuracy: 0.242236 | 3.679 sec/iter
Average sparseness: 0.9821428571428572
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.98214285

Epoch: 69 | Batch: 000 / 003 | Total loss: 8.895 | Reg loss: 0.015 | Tree loss: 8.895 | Accuracy: 0.253906 | 3.683 sec/iter
Epoch: 69 | Batch: 001 / 003 | Total loss: 8.876 | Reg loss: 0.015 | Tree loss: 8.876 | Accuracy: 0.234375 | 3.683 sec/iter
Epoch: 69 | Batch: 002 / 003 | Total loss: 8.857 | Reg loss: 0.015 | Tree loss: 8.857 | Accuracy: 0.198758 | 3.68 sec/iter
Average sparseness: 0.9821428571428572
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
layer 9: 0.9821428571428573
layer 10: 0.9821428571428573
Epoch: 70 | Batch: 000 / 003 | Total loss: 8.872 | Reg loss: 0.015 | Tree loss: 8.872 | Accuracy: 0.246094 | 3.683 sec/iter
Epoch: 70 | Batch: 001 / 003 | Total loss: 8.832 | Reg loss: 0.015 | Tree loss: 8.832 | Accuracy: 0.234375 | 3.683 sec/iter
Epoch: 70 | Batch: 002 / 003 | Tot

Average sparseness: 0.9821428571428572
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
layer 9: 0.9821428571428573
layer 10: 0.9821428571428573
Epoch: 81 | Batch: 000 / 003 | Total loss: 8.444 | Reg loss: 0.017 | Tree loss: 8.444 | Accuracy: 0.242188 | 3.682 sec/iter
Epoch: 81 | Batch: 001 / 003 | Total loss: 8.419 | Reg loss: 0.017 | Tree loss: 8.419 | Accuracy: 0.220703 | 3.682 sec/iter
Epoch: 81 | Batch: 002 / 003 | Total loss: 8.377 | Reg loss: 0.017 | Tree loss: 8.377 | Accuracy: 0.229814 | 3.68 sec/iter
Average sparseness: 0.9821428571428572
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.982142857

Epoch: 92 | Batch: 000 / 003 | Total loss: 7.985 | Reg loss: 0.018 | Tree loss: 7.985 | Accuracy: 0.244141 | 3.682 sec/iter
Epoch: 92 | Batch: 001 / 003 | Total loss: 7.962 | Reg loss: 0.018 | Tree loss: 7.962 | Accuracy: 0.205078 | 3.682 sec/iter
Epoch: 92 | Batch: 002 / 003 | Total loss: 7.943 | Reg loss: 0.018 | Tree loss: 7.943 | Accuracy: 0.195652 | 3.68 sec/iter
Average sparseness: 0.9821428571428572
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
layer 9: 0.9821428571428573
layer 10: 0.9821428571428573
Epoch: 93 | Batch: 000 / 003 | Total loss: 7.948 | Reg loss: 0.018 | Tree loss: 7.948 | Accuracy: 0.220703 | 3.682 sec/iter
Epoch: 93 | Batch: 001 / 003 | Total loss: 7.936 | Reg loss: 0.018 | Tree loss: 7.936 | Accuracy: 0.205078 | 3.682 sec/iter
Epoch: 93 | Batch: 002 / 003 | Tot

In [31]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [33]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 11.999511480214949


# Extract Rules

# Accumulate samples in the leaves

In [34]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 4094


In [35]:
method = 'greedy'

In [36]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [37]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Pattern 25

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))


============== Pattern 160 ==============
============== Pattern 161 ==============
============== Pattern 162 ==============
============== Pattern 163 ==============
============== Pattern 164 ==============
============== Pattern 165 ==============
============== Pattern 166 ==============
============== Pattern 167 ==============
============== Pattern 168 ==============
============== Pattern 169 ==============
============== Pattern 170 ==============
============== Pattern 171 ==============
============== Pattern 172 ==============
============== Pattern 173 ==============
============== Pattern 174 ==============
============== Pattern 175 ==============
============== Pattern 176 ==============
============== Pattern 177 ==============
============== Pattern 178 ==============
============== Pattern 179 ==============
============== Pattern 180 ==============
============== Pattern 181 ==============
============== Pattern 182 ==============
============== Pattern 183 =======

============== Pattern 530 ==============
============== Pattern 531 ==============
============== Pattern 532 ==============
============== Pattern 533 ==============
============== Pattern 534 ==============
============== Pattern 535 ==============
============== Pattern 536 ==============
============== Pattern 537 ==============
============== Pattern 538 ==============
============== Pattern 539 ==============
============== Pattern 540 ==============
============== Pattern 541 ==============
============== Pattern 542 ==============
============== Pattern 543 ==============
============== Pattern 544 ==============
============== Pattern 545 ==============
============== Pattern 546 ==============
============== Pattern 547 ==============
============== Pattern 548 ==============
============== Pattern 549 ==============
============== Pattern 550 ==============
============== Pattern 551 ==============
============== Pattern 552 ==============
============== Pattern 553 =======

============== Pattern 969 ==============
============== Pattern 970 ==============
============== Pattern 971 ==============
============== Pattern 972 ==============
============== Pattern 973 ==============
============== Pattern 974 ==============
============== Pattern 975 ==============
============== Pattern 976 ==============
============== Pattern 977 ==============
============== Pattern 978 ==============
============== Pattern 979 ==============
============== Pattern 980 ==============
============== Pattern 981 ==============
============== Pattern 982 ==============
============== Pattern 983 ==============
============== Pattern 984 ==============
============== Pattern 985 ==============
============== Pattern 986 ==============
============== Pattern 987 ==============
============== Pattern 988 ==============
============== Pattern 989 ==============
============== Pattern 990 ==============
============== Pattern 991 ==============
============== Pattern 992 =======

============== Pattern 1638 ==============
============== Pattern 1639 ==============
============== Pattern 1640 ==============
============== Pattern 1641 ==============
============== Pattern 1642 ==============
============== Pattern 1643 ==============
============== Pattern 1644 ==============
============== Pattern 1645 ==============
============== Pattern 1646 ==============
============== Pattern 1647 ==============
============== Pattern 1648 ==============
============== Pattern 1649 ==============
============== Pattern 1650 ==============
============== Pattern 1651 ==============
============== Pattern 1652 ==============
============== Pattern 1653 ==============
============== Pattern 1654 ==============
============== Pattern 1655 ==============
============== Pattern 1656 ==============
============== Pattern 1657 ==============
============== Pattern 1658 ==============
============== Pattern 1659 ==============
============== Pattern 1660 ==============
===========

============== Pattern 2391 ==============
============== Pattern 2392 ==============
============== Pattern 2393 ==============
============== Pattern 2394 ==============
============== Pattern 2395 ==============
============== Pattern 2396 ==============
============== Pattern 2397 ==============
============== Pattern 2398 ==============
============== Pattern 2399 ==============
============== Pattern 2400 ==============
============== Pattern 2401 ==============
============== Pattern 2402 ==============
============== Pattern 2403 ==============
============== Pattern 2404 ==============
============== Pattern 2405 ==============
============== Pattern 2406 ==============
============== Pattern 2407 ==============
============== Pattern 2408 ==============
============== Pattern 2409 ==============
============== Pattern 2410 ==============
============== Pattern 2411 ==============
============== Pattern 2412 ==============
============== Pattern 2413 ==============
===========

============== Pattern 2829 ==============
============== Pattern 2830 ==============
============== Pattern 2831 ==============
============== Pattern 2832 ==============
============== Pattern 2833 ==============
============== Pattern 2834 ==============
============== Pattern 2835 ==============
============== Pattern 2836 ==============
============== Pattern 2837 ==============
============== Pattern 2838 ==============
============== Pattern 2839 ==============
============== Pattern 2840 ==============
============== Pattern 2841 ==============
============== Pattern 2842 ==============
============== Pattern 2843 ==============
============== Pattern 2844 ==============
============== Pattern 2845 ==============
============== Pattern 2846 ==============
============== Pattern 2847 ==============
============== Pattern 2848 ==============
============== Pattern 2849 ==============
============== Pattern 2850 ==============
============== Pattern 2851 ==============
===========

============== Pattern 3781 ==============
============== Pattern 3782 ==============
============== Pattern 3783 ==============
============== Pattern 3784 ==============
============== Pattern 3785 ==============
============== Pattern 3786 ==============
============== Pattern 3787 ==============
============== Pattern 3788 ==============
============== Pattern 3789 ==============
============== Pattern 3790 ==============
============== Pattern 3791 ==============
============== Pattern 3792 ==============
============== Pattern 3793 ==============
============== Pattern 3794 ==============
============== Pattern 3795 ==============
============== Pattern 3796 ==============
============== Pattern 3797 ==============
============== Pattern 3798 ==============
============== Pattern 3799 ==============
============== Pattern 3800 ==============
============== Pattern 3801 ==============
============== Pattern 3802 ==============
============== Pattern 3803 ==============
===========